In [ ]:
%%bash

# Instal Java
apt-get update --fix-missing
apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
pip install -q pyspark

In [12]:
import os
import time
import random
from datetime import datetime, timedelta
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col, sum, max

os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enable", True)

In [20]:
# -----------------------------
# Parâmetros
# -----------------------------
n_clientes = 200
n_transacoes = 1000

nomes = ["Joao", "Luiza", "Gabriela", "Rafael", "Jean", "Daniela", "Paulo", "Mariana", "Felipe", "Carla", "Tiago", "Fernanda"]
estados = ["SP", "RJ", "MG", "PR", "RS", "PE", "BA", "PB", "GO", "DF"]
cidades = {
    "SP": ["São Paulo", "Campinas", "São Caetano"],
    "RJ": ["Rio de Janeiro", "Niterói", "Cabo Frio"],
    "MG": ["Belo Horizonte", "Betim", "Uberlândia"],
    "PR": ["Curitiba", "Londrina", "Maringá"],
    "RS": ["Porto Alegre", "Caxias do Sul"],
    "PE": ["Recife", "Olinda"],
    "BA": ["Salvador", "Feira de Santana"],
    "PB": ["João Pessoa", "Campina Grande"],
    "GO": ["Goiânia", "Anápolis"],
    "DF": ["Brasília"]
}

In [21]:
# -----------------------------
# Criar clientes
# -----------------------------
clientes = []
for i in range(1, n_clientes + 1):
    nome = random.choice(nomes)
    sexo = random.choice(["M", "F"])
    estado = random.choice(estados)
    cidade = random.choice(cidades[estado])
    clientes.append((i, nome, sexo, estado, cidade))

In [22]:
# -----------------------------
# Criar contas
# -----------------------------
contas = {}
base_timestamp = int(time.mktime(datetime(2021, 1, 1).timetuple()))
for cliente_id, _, _, _, _ in clientes:
    id_conta = 190000 + cliente_id
    dt_criacao = base_timestamp + random.randint(0, 60*60*24*365)  # dentro de 1 ano
    contas[cliente_id] = (id_conta, dt_criacao)

In [23]:
# -----------------------------
# Criar produtos
# -----------------------------
produtos = []
for _, (id_conta, _) in contas.items():
    produtos.append({
        "conta_id": id_conta,
        "fl_cartao_credito": random.choice([0, 1]),
        "fl_investidor": random.choice([0, 1]),
        "fl_credito": random.choice([0, 1]),
    })


In [24]:
# -----------------------------
# Criar transações
# -----------------------------
transacoes = []
for _ in range(n_transacoes):
    conta_id = random.choice(list(contas.values()))[0]
    valor = round(random.uniform(1, 5000), 2)
    data_transacao = datetime(2021, 1, 1) + timedelta(days=random.randint(0, 365),
                                                      hours=random.randint(0, 23),
                                                      minutes=random.randint(0, 59))
    transacoes.append({
        "conta_id": conta_id,
        "valor": valor,
        "dt_transacao": data_transacao.strftime("%Y-%m-%d %H:%M:%S")
    })

In [18]:
# -----------------------------
# Criando DataFrames Spark
# -----------------------------
df_clientes = spark.createDataFrame(clientes, schema=['id', 'nome', 'sexo', 'estado', 'cidade'])

df_contas = (
    spark.createDataFrame([Row(k, *v) for k, v in contas.items()],
                          schema=['cliente_id', 'conta_id', 'dt_criacao'])
    .withColumn('dt_criacao', col('dt_criacao').cast('timestamp'))
    .withColumnRenamed('dt_criacao', 'dt_criacao_novo')
)

df_produtos = spark.createDataFrame(produtos)

df_transacoes = (
    spark.createDataFrame(transacoes)
    .withColumn('dt_transacao', col('dt_transacao').cast('timestamp'))
)

In [19]:
# -----------------------------
# Conferindo
# -----------------------------
print("Clientes:", df_clientes.count())
print("Contas:", df_contas.count())
print("Produtos:", df_produtos.count())
print("Transações:", df_transacoes.count())

df_clientes.show(5, truncate=False)
df_contas.show(5, truncate=False)
df_produtos.show(5, truncate=False)
df_transacoes.show(5, truncate=False)

Clientes: 200
Contas: 200
Produtos: 200
Transações: 1000
+---+--------+----+------+-------------+
|id |nome    |sexo|estado|cidade       |
+---+--------+----+------+-------------+
|1  |Daniela |F   |SP    |São Paulo    |
|2  |Rafael  |F   |RS    |Caxias do Sul|
|3  |Gabriela|M   |RJ    |Cabo Frio    |
|4  |Tiago   |F   |GO    |Goiânia      |
|5  |Gabriela|M   |SP    |Campinas     |
+---+--------+----+------+-------------+
only showing top 5 rows

+----------+--------+-------------------+
|cliente_id|conta_id|dt_criacao_novo    |
+----------+--------+-------------------+
|1         |190001  |2021-04-01 18:06:33|
|2         |190002  |2021-04-16 07:16:28|
|3         |190003  |2021-04-05 20:49:41|
|4         |190004  |2021-11-24 19:44:25|
|5         |190005  |2021-10-04 08:20:39|
+----------+--------+-------------------+
only showing top 5 rows

+--------+-----------------+----------+-------------+
|conta_id|fl_cartao_credito|fl_credito|fl_investidor|
+--------+-----------------+----------

In [ ]:
# Comandos para exibir os dados

df_clientes.show()
df_contas.show()
df_produtos.show()
df_transacoes.show()

Exercicios - Praticas Spark

In [27]:
# filtrando dados
# Exibir o valor
clientes_cartao = df_produtos.filter(col("fl_cartao_credito") == 1)
clientes_cartao.show()

+--------+-----------------+----------+-------------+
|conta_id|fl_cartao_credito|fl_credito|fl_investidor|
+--------+-----------------+----------+-------------+
|  190004|                1|         1|            0|
|  190007|                1|         0|            0|
|  190008|                1|         0|            0|
|  190010|                1|         1|            0|
|  190011|                1|         1|            0|
|  190017|                1|         1|            1|
|  190018|                1|         0|            0|
|  190020|                1|         0|            0|
|  190022|                1|         0|            1|
|  190023|                1|         0|            1|
|  190024|                1|         1|            1|
|  190026|                1|         0|            1|
|  190028|                1|         1|            0|
|  190029|                1|         0|            1|
|  190030|                1|         0|            1|
|  190031|                1|

In [30]:
df_transacoes.show()

+--------+-------------------+-------+
|conta_id|       dt_transacao|  valor|
+--------+-------------------+-------+
|  190093|2021-12-03 19:29:00|3585.58|
|  190174|2021-10-04 23:07:00| 351.75|
|  190036|2021-05-17 11:00:00|4005.24|
|  190152|2021-01-15 07:53:00|4887.74|
|  190003|2021-04-10 18:48:00| 781.94|
|  190109|2021-06-10 23:05:00|2318.13|
|  190073|2021-03-11 07:54:00|1936.97|
|  190021|2021-12-31 23:04:00|2301.97|
|  190097|2021-03-23 23:35:00|  434.6|
|  190038|2021-05-01 11:48:00|4186.17|
|  190088|2021-09-29 06:44:00|3977.46|
|  190155|2021-01-06 20:46:00|3616.36|
|  190083|2021-12-13 20:35:00| 932.63|
|  190198|2021-03-01 12:31:00|4540.47|
|  190184|2021-10-10 22:43:00|2574.42|
|  190196|2021-02-25 20:17:00|1397.47|
|  190159|2021-10-02 21:08:00|2939.37|
|  190036|2021-01-03 18:28:00|2238.92|
|  190083|2021-06-04 09:44:00| 2378.8|
|  190185|2021-09-28 07:36:00|3271.31|
+--------+-------------------+-------+
only showing top 20 rows



In [33]:
# Realizando Joins
# Exibir Valor
resultado = df_transacoes.join(clientes_cartao, on= "conta_id", how='left') \
                         .filter(col("valor") > 100) \
                         .select("conta_id", "valor", "dt_transacao")
resultado.show()

+--------+-------+-------------------+
|conta_id|  valor|       dt_transacao|
+--------+-------+-------------------+
|  190002|4806.85|2021-07-27 15:12:00|
|  190003| 781.94|2021-04-10 18:48:00|
|  190021|2301.97|2021-12-31 23:04:00|
|  190031|4623.37|2021-03-03 13:07:00|
|  190036|4005.24|2021-05-17 11:00:00|
|  190036|2238.92|2021-01-03 18:28:00|
|  190038|4186.17|2021-05-01 11:48:00|
|  190048|1334.98|2021-05-16 14:19:00|
|  190058| 862.74|2021-04-10 03:43:00|
|  190062|4143.24|2021-08-06 15:57:00|
|  190065| 355.92|2021-02-19 14:11:00|
|  190073|1936.97|2021-03-11 07:54:00|
|  190083| 932.63|2021-12-13 20:35:00|
|  190083| 2378.8|2021-06-04 09:44:00|
|  190088|3977.46|2021-09-29 06:44:00|
|  190093|3585.58|2021-12-03 19:29:00|
|  190097|  434.6|2021-03-23 23:35:00|
|  190109|2318.13|2021-06-10 23:05:00|
|  190111| 4629.3|2021-12-05 19:21:00|
|  190119|2098.88|2021-06-05 04:30:00|
+--------+-------+-------------------+
only showing top 20 rows



In [40]:
# Encontrando a maior transação & Exibir o valor
# Aplicando alias
maior_transacao = df_transacoes.agg(max("valor").alias("max_valor")).collect()[0]["max_valor"]
maior_transacao

4999.23


In [42]:
maior_transacao

4999.23

In [43]:
# Realize select de colunas especificas dos seus dataframes - .select()
sel_transacoes = df_transacoes.select("valor")
sel_transacoes.show()

+-------+
|  valor|
+-------+
|3585.58|
| 351.75|
|4005.24|
|4887.74|
| 781.94|
|2318.13|
|1936.97|
|2301.97|
|  434.6|
|4186.17|
|3977.46|
|3616.36|
| 932.63|
|4540.47|
|2574.42|
|1397.47|
|2939.37|
|2238.92|
| 2378.8|
|3271.31|
+-------+
only showing top 20 rows



Exercicios - Praticas Spark SQL

In [44]:
# Criação de viewTemporaria
df_clientes.createOrReplaceTempView('CLIENTES')
sel_clientes = spark.sql(f'''
                          SELECT count(nome), estado
                          FROM CLIENTES
                          group by estado
                        ''')

sel_clientes.show()

+-----------+------+
|count(nome)|estado|
+-----------+------+
|         14|    SC|
|         26|    GO|
|         22|    SP|
|         16|    RS|
|         34|    MG|
|         19|    BA|
|         19|    PE|
|         19|    RJ|
|         13|    DF|
|         18|    PR|
+-----------+------+



In [ ]:
# Criar as views dos demais dfs
# df_clientes OK
# Para casa
# df_contas.show()
# df_produtos.show()
# df_transacoes.show()

In [ ]:
# filtrando dados .sql() - Where
# Exibir o valor


In [ ]:
# Realizando Joins
# Exibir Valor

In [ ]:
# Encontrando o maior valor da transação & Exibir o valor com spark SQL
# Aplicando alias

Quantidade de Clientes por Estado

In [45]:
contagem_cliente_estado = df_clientes.groupBy("estado").count()
print(contagem_cliente_estado.show())

+------+-----+
|estado|count|
+------+-----+
|    SC|   14|
|    GO|   26|
|    SP|   22|
|    RS|   16|
|    MG|   34|
|    BA|   19|
|    PE|   19|
|    RJ|   19|
|    DF|   13|
|    PR|   18|
+------+-----+

None


Cliente e Conta

In [47]:
juncao = df_contas.join(
    df_clientes,
    df_contas["cliente_id"] == df_clientes["id"],
    how="inner"
).select(
    df_clientes["id"].alias("cliente_id"),
    df_contas["conta_id"]
)
juncao.show()

+----------+--------+
|cliente_id|conta_id|
+----------+--------+
|        26|  190026|
|        29|  190029|
|        65|  190065|
|        19|  190019|
|        54|  190054|
|        22|  190022|
|         7|  190007|
|        77|  190077|
|        34|  190034|
|        50|  190050|
|        94|  190094|
|        57|  190057|
|        32|  190032|
|        43|  190043|
|        84|  190084|
|        31|  190031|
|        39|  190039|
|        98|  190098|
|        25|  190025|
|        95|  190095|
+----------+--------+
only showing top 20 rows



Soma das transações por cliente e conta

In [57]:
df_contas_clientes = df_contas.join(df_clientes,
                                    df_contas.cliente_id == df_clientes.id, how='inner') \
                              .select(df_contas.conta_id, df_clientes.nome)

df_transacao = df_transacoes.join(df_contas_clientes, 'conta_id', how='inner')

df = df_transacao.groupBy('nome', 'conta_id') \
                                     .agg(sum('valor').alias('soma'))

df.orderBy(df.soma.desc()).show()

+--------+--------+------------------+
|    nome|conta_id|              soma|
+--------+--------+------------------+
|Fernanda|  190020|37227.829999999994|
|   Carla|  190036|           36256.7|
|   Tiago|  190172|34861.020000000004|
|Gabriela|  190034|27612.219999999998|
|Fernanda|  190076|27115.730000000003|
|    Jean|  190072|          26860.76|
| Mariana|  190162|          26717.71|
|   Tiago|  190127|          25828.42|
| Daniela|  190050|          25380.27|
|Fernanda|  190059|          25357.56|
|Gabriela|  190155|          25074.25|
|  Rafael|  190035|          24860.19|
|Gabriela|  190091|          24821.93|
|    Jean|  190049|          24742.57|
| Daniela|  190056|          24318.62|
|  Rafael|  190132|           23548.4|
|   Paulo|  190098|23414.719999999998|
| Daniela|  190195|23361.949999999997|
|  Felipe|  190186|22904.850000000006|
|  Felipe|  190125|          22881.56|
+--------+--------+------------------+
only showing top 20 rows

